In [1]:
import numpy as np
import pandas as pd
from pandas_summary import DataFrameSummary

import matplotlib as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 

init_notebook_mode(connected=True)
%matplotlib inline 

# Loading Data 

In [10]:
ts_covid_confirmed = pd.read_csv(
    '/Users/andang/Documents/workspace/private/python/COVID-19/data/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
ts_covid_deaths = pd.read_csv(
    '/Users/andang/Documents/workspace/private/python/COVID-19/data/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')
ts_covid_recovered = pd.read_csv(
    '/Users/andang/Documents/workspace/private/python/COVID-19/data/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv')

daily_report = pd.read_csv(
    '/Users/andang/Documents/workspace/private/python/COVID-19/data/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/03-15-2020.csv')

In [5]:
ts_covid_recovered.head(1)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20
0,NaN,Thailand,15.0,101.0,0,0,0,0,2,2,...,31,31,31,31,33,34,34,35,35,35


# Functions

# Preprocessing

In [68]:
def sum_china(df):
    return df.groupby(df['Country/Region'], as_index=False).sum()


def header_to_row(df):
    return df.melt(id_vars=['Country/Region', 'Lat', 'Long'],
                   var_name='Date', value_name='Value').sort_values(by='Value', ascending=False)


def top_10_countries(df):
    grouped_df = df.groupby(df['Country/Region'],
               as_index=False).sum().sort_values(by='Value', ascending=False)[:10]
    
    return grouped_df['Country/Region'].values.tolist()

In [70]:
# Transformation
transformed_covod_recov = header_to_row(sum_china(ts_covid_recovered))

countries = top_10_countries(transformed_covod_recov)

# Statistics

In [67]:
transformed_covod_recov.groupby(
    transformed_covod_recov['Country/Region'], as_index=False).sum().sort_values(by='Value', ascending=False)[:10]

,Country/Region,Lat,Long,Value
28,China,58500.1818,198958.6638,1221915
66,Iran,1728.0000,2862.0000,28609
70,Italy,2322.0000,648.0000,12239
77,"Korea, South",1944.0000,6912.0000,3359
122,Singapore,69.2982,5606.9982,1979
35,Cruise Ship,1913.9598,7540.4520,1919
126,Spain,2160.0000,-216.0000,1759
72,Japan,1944.0000,7452.0000,1506
133,Thailand,810.0000,5454.0000,899
85,Malaysia,135.0000,6075.0000,607


# Plots

In [73]:
fig = go.Figure()

for country in countries:
    recov_country = transformed_covod_recov[transformed_covod_recov['Country/Region'] == country]
    
    fig.add_trace(go.Scatter(
        x=recov_country.Date,
        y=recov_country.Value,
        name=country,
        opacity=0.8))
fig['layout']['xaxis']['autorange'] = "reversed"

fig.show()